In [3]:
import mne
import numpy as np
import pandas as pd
import pathlib
import scipy

In [4]:
BASE_DIR = pathlib.Path(r'C:\Users\User\Desktop\Rambam collaboration\data')
print(BASE_DIR)

C:\Users\User\Desktop\Rambam collaboration\data


In [5]:
def extract_spectral_features(raw):
    # Initialize an empty list to store features
    features = []
    x = raw.get_data()
    # Calculate spectral features for each channel
    freqs, psd_array = scipy.signal.csd(x, x, fs=raw.info['sfreq'])
    # Loop through each channel
# for i, ch in enumerate(raw.info['ch_names']):
    alpha_band = np.logical_and(freqs >= 8, freqs <= 13)
    delta_band = np.logical_and(freqs >= 0.5, freqs <= 4)
    
    alpha_power = np.mean(psd_array[:, alpha_band], axis=1)
    delta_power = np.mean(psd_array[:, delta_band], axis=1)

    spectral_edge = freqs[np.argmax(np.cumsum(psd_array, axis=1) >= 0.9, axis=1)]
    spectral_entropy = -np.sum(psd_array * np.log2(psd_array), axis=1)

    # Add spectral features for this channel to the list
    features.extend([alpha_power, delta_power, spectral_edge, spectral_entropy])

    return np.array(features).flatten()

In [6]:
# Loop through each patient (009, 015, 033, 045)
for patient_id in ['045', '033', '009', '015']:
    print("Patient: " + patient_id)
    # List of (file type, label) tuples
    files = [('base', 0), ('surgery', 1)]

    for file_type, label in files:
        # Load the file
        file_path = BASE_DIR / f'{patient_id}/{patient_id}_{file_type}.vhdr'

        try:
            raw = mne.io.read_raw(file_path, preload=True)
        except ValueError as e:
            print(f"Error reading {file_path}: {e}")
            continue

        print(raw.ch_names)
        raw.pick(['CP5', 'CP6', 'F7', 'F8', 'FC5', 'FC6', 'P7', 'P8', 'T7', 'T8'])
        raw.filter(l_freq=0.1, h_freq=70)
        raw.notch_filter(50)
        raw.set_montage('standard_1020', on_missing='ignore')

        # Divide into 10-20s windows
        window_size = 30
        overlap = 15
        data = []

        for start in range(0, int(len(raw) / raw.info['sfreq']) - window_size, overlap):
            window_raw = raw.copy().crop(tmin=start, tmax=start + window_size, include_tmax=False)
            # print(window_raw.shape)
            window_features = extract_spectral_features(window_raw)
            print(window_features.shape)
            # features_doubled = np.hstack([window_features, window_features])  # Double the features
            data.append(np.hstack([window_features, label]))  # Append features and label to data

        # Generate the column names
        feature_names = [f'{ch}_{feature}' for ch in raw.info['ch_names'] for feature in ['alpha_power', 'delta_power', 'spectral_edge', 'spectral_entropy']]
        columns = feature_names + ['label']
        
        # Create the DataFrame
        df = pd.DataFrame(data, columns=columns)
        
        try:
            df.to_csv(f'{patient_id}_{file_type}_features.csv', index=False)
        except Exception as e:
            print(f"Error saving file {patient_id}_{file_type}_features.csv: {e}")


Patient: 045
Extracting parameters from C:\Users\User\Desktop\Rambam collaboration\data\045\045_base.vhdr...
Setting channel info structure...


C:\Users\User\AppData\Local\Temp\ipykernel_17992\115966606.py:12: RuntimeWarning: Online software filter detected. Using software filter settings and ignoring hardware values
  raw = mne.io.read_raw(file_path, preload=True)


Reading 0 ... 137029  =      0.000 ...   274.058 secs...
['FP1', 'FP2', 'F7', 'F8', 'FC5', 'FZ', 'FC6', 'T8', 'T7', 'CZ', 'CP6', 'CP5', 'P7', 'PZ', 'P8', 'OZ', 'REF', 'GRN']
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 70 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 70.00 Hz
- Upper transition bandwidth: 17.50 Hz (-6 dB cutoff frequency: 78.75 Hz)
- Filter length: 16501 samples (33.002 s)

Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 49 - 51 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window wit

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished


(40,)
(40,)
(40,)
(40,)
(40,)
(40,)
(40,)
(40,)
(40,)
(40,)
(40,)
(40,)
(40,)
(40,)
(40,)
(40,)
(40,)
Extracting parameters from C:\Users\User\Desktop\Rambam collaboration\data\045\045_surgery.vhdr...
Setting channel info structure...
Reading 0 ... 5859379  =      0.000 ... 11718.758 secs...


C:\Users\User\AppData\Local\Temp\ipykernel_17992\115966606.py:12: RuntimeWarning: Online software filter detected. Using software filter settings and ignoring hardware values
  raw = mne.io.read_raw(file_path, preload=True)


KeyboardInterrupt: 